In [1]:
import numpy as np
import pandas as pd
import re
import json

In [2]:
data = pd.read_csv('data.csv')
data.head(3)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [4]:
data.columns

Index(['status', 'private pool', 'propertyType', 'street', 'baths',
       'homeFacts', 'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds',
       'state', 'stories', 'mls-id', 'PrivatePool', 'MlsId', 'target'],
      dtype='object')

## EDA

#### Признак "target"

In [5]:
data.isna().sum()/(data.shape[0]/100)

status          10.583136
private pool    98.891525
propertyType     9.208479
street           0.000530
baths           28.192532
homeFacts        0.000000
fireplace       72.661956
city             0.009014
schools          0.000000
sqft            10.757851
zipcode          0.000000
beds            24.200856
state            0.000000
stories         39.957846
mls-id          93.387330
PrivatePool     89.312672
MlsId           17.731352
target           0.657767
dtype: float64

In [6]:
data.drop(data[data.target.isna()].index, inplace=True)

In [7]:
import re
def make_target(x):
    x = re.sub('[^0-9]', '', x)
    x = int(x)
    return x

In [8]:
data.target = data.target.apply(make_target)
data.target.describe()

count    3.747040e+05
mean     6.447575e+05
std      1.841431e+06
min      1.000000e+00
25%      1.869900e+05
50%      3.200000e+05
75%      5.821000e+05
max      1.950000e+08
Name: target, dtype: float64

In [9]:
data = data.reset_index(drop=True)

In [10]:
cols_drop = ['street', 'mls-id', 'MlsId', 'zipcode']
data.drop(cols_drop, axis=1, inplace=True)

In [11]:
df = data.copy()

In [12]:
df.shape

(374704, 14)

#### Признак "homeFacts"

In [13]:
df['homeFacts'] = df['homeFacts'].apply(lambda x: x.replace('None', '0'))

In [14]:
# df['homeFacts'].value_counts()

In [15]:
# df['homeFacts'].value_counts().index.tolist()

In [16]:
# Для замены апострофов в словах
change_old = ["Attch'd", "Addtn'l",'"closet"', "Hennen\'s", "Penske's","Req'd"]
change_new = ['Attch`d','Addtn`l','`closet`', 'Hennen`s', 'Penske`s','Req`d']

In [17]:
for i in range(0,len(change_old)):
    df['homeFacts'] = df['homeFacts'].apply(lambda x: x.replace(change_old[i],change_new[i]))

In [18]:
df['homeFacts'] = df['homeFacts'].apply(lambda x: x.replace("'",'"'))

In [49]:
ss = df['homeFacts'][4]
sa = json.loads(ss)
sa

{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'},
  {'factValue': '', 'factLabel': 'Remodeled year'},
  {'factValue': '', 'factLabel': 'Heating'},
  {'factValue': '', 'factLabel': 'Cooling'},
  {'factValue': '', 'factLabel': 'Parking'},
  {'factValue': '10,019 sqft', 'factLabel': 'lotsize'},
  {'factValue': 0, 'factLabel': 'Price/sqft'}]}

In [19]:
def app(x,k):
    js = json.loads(x)
    return js['atAGlanceFacts'][k]['factValue']

In [20]:
cols_homeFacts = ['Year built','Remodeled year','Heating','Cooling','Parking','lotsize','Price/sqft']
df[cols_homeFacts] = ''

In [21]:
df['Year built'] = df['homeFacts'].apply(lambda x: app(x,0))
df['Remodeled year'] = df['homeFacts'].apply(lambda x: app(x,1))
df['Heating'] = df['homeFacts'].apply(lambda x: app(x,2))
df['Cooling'] = df['homeFacts'].apply(lambda x: app(x,3))
df['Parking'] = df['homeFacts'].apply(lambda x: app(x,4))
df['lotsize'] = df['homeFacts'].apply(lambda x: app(x,5))
df['Price/sqft'] = df['homeFacts'].apply(lambda x: app(x,6))

In [54]:
df['Year built']

0         2019
1         2019
2         1961
3         2006
4             
          ... 
374699    1990
374700    1924
374701    1950
374702       0
374703    2019
Name: Year built, Length: 374704, dtype: object

In [60]:
df['Year built'].value_counts()

        58521
2019    32142
2006     7954
2005     7449
2007     7096
        ...  
1057        1
1843        1
1845        1
1812        1
1823        1
Name: Year built, Length: 229, dtype: int64

In [59]:
df['Remodeled year'].value_counts()

        198528
0        26341
2006      5493
2005      4777
2007      4355
         ...  
1111         1
1869         1
1888         1
1845         1
1853         1
Name: Remodeled year, Length: 155, dtype: int64

In [55]:
df['lotsize'] 

0                   0
1           5828 sqft
2          8,626 sqft
3          8,220 sqft
4         10,019 sqft
             ...     
374699     8,500 sqft
374700               
374701     1,600 sqft
374702              0
374703     6,969 sqft
Name: lotsize, Length: 374704, dtype: object

#### Признак "schools"

In [22]:
df['schools']=data['schools']

In [23]:
df['schools'] = df['schools'].apply(lambda x: x.replace('None', '0'))

In [24]:
# df['schools'].value_counts()

In [25]:
# df['schools'].value_counts().index.tolist()

In [26]:
# Просмотр строк с апострофом среди текста, например, "Lil\' Wonders Development Center Private School" 
ss = df['schools'].loc[1485]
ss

'[{\'rating\': [\'6\', \'6\', \'7\', \'7\', \'5\', \'9\', \'NR\', \'NR\'], \'data\': {\'Distance\': [\'1.8 mi\', \'4.6 mi\', \'1.9 mi\', \'4.6 mi\', \'4.7 mi\', \'7.0 mi\', \'4.1 mi\', \'4.1 mi\'], \'Grades\': [\'PK–5\', \'6–8\', \'9–12\', \'PK–5\', \'K–12\', \'9–12\', \'PK–8\', \'PK–5\']}, \'name\': [\'Gary D Wright Elementary School\', \'Hampshire Middle School\', \'Hampshire High School\', \'Hampshire Elementary School\', \'Cambridge Lake Charter School\', \'Harry D Jacobs High School\', \'St Charles Borromeo Catholic Private School\', "Lil\' Wonders Development Center Private School"]}]'

In [27]:
old = ["O\'", " O`,",  "'s", "\'Duke\'", "B\'Nai", "B\'inai",  "L\'Ouver", "L\'ouver",  "L\'Banos", "L\'Dovid", 
              "Lil\'", "D\'",  "d\'Evelyn", "Q\'", "Loggers\'", "Teach N\'",  "K\'Tana", "R&D`",  "Citizens\'",  "Kids\'", 
              "Girls\' ", "d\'Chassidei", "d\'Gur", "d\'Rav", "Pe\'Er", "Be\'Er", "Kids \'R\' Kids", "\'joe\' ", 
              "Meadow\'S", "Sheriff\'S", "Friends\' ", "\'Mother\'", "Int\'L", "\'buddy\'"]
new = ['O`',  " O\', ", '`s', '`Duke`', 'B`Nai', 'B`inai',  'L`Ouver', 'L`Ouver', 'L`Banos', 'L`Dovid', 
              'Lil`', 'D`', 'd`Evelyn', 'Q`', 'Loggers`', 'Teach N`', 'K`Tana', "R&D\'",   'Citizens`', 'Kids`', 
              'Girls` ',  'd`Chassidei','d`Gur', 'd`Rav', 'Pe`Er', 'Be`Er', 'Kids `R` Kids', '`joe` ', 
              'Meadow`S', 'Sheriff`S', 'Friends` ', '`Mother`', 'Int`L', '`buddy`']

In [28]:
for i in range(0,len(old)):
    df['schools'] = df['schools'].apply(lambda x: x.replace(old[i], new[i]))

In [30]:
df['schools'] = df['schools'].apply(lambda x: x.replace("'",'"'))

In [46]:
# Проверка после преобразования отдельных строк с неправильными данными
ss = df['schools'].loc[1485]
sa = json.loads(ss)
sa

[{'rating': ['6', '6', '7', '7', '5', '9', 'NR', 'NR'],
  'data': {'Distance': ['1.8 mi',
    '4.6 mi',
    '1.9 mi',
    '4.6 mi',
    '4.7 mi',
    '7.0 mi',
    '4.1 mi',
    '4.1 mi'],
   'Grades': ['PK–5', '6–8', '9–12', 'PK–5', 'K–12', '9–12', 'PK–8', 'PK–5']},
  'name': ['Gary D Wright Elementary School',
   'Hampshire Middle School',
   'Hampshire High School',
   'Hampshire Elementary School',
   'Cambridge Lake Charter School',
   'Harry D Jacobs High School',
   'St Charles Borromeo Catholic Private School',
   'Lil` Wonders Development Center Private School']}]

In [50]:
ss = df['schools'].loc[1485][1:len(df['schools'][1485])-1]
sa = json.loads(ss)
sa

{'rating': ['6', '6', '7', '7', '5', '9', 'NR', 'NR'],
 'data': {'Distance': ['1.8 mi',
   '4.6 mi',
   '1.9 mi',
   '4.6 mi',
   '4.7 mi',
   '7.0 mi',
   '4.1 mi',
   '4.1 mi'],
  'Grades': ['PK–5', '6–8', '9–12', 'PK–5', 'K–12', '9–12', 'PK–8', 'PK–5']},
 'name': ['Gary D Wright Elementary School',
  'Hampshire Middle School',
  'Hampshire High School',
  'Hampshire Elementary School',
  'Cambridge Lake Charter School',
  'Harry D Jacobs High School',
  'St Charles Borromeo Catholic Private School',
  'Lil` Wonders Development Center Private School']}

In [51]:
ss = df['schools'].loc[1492][1:len(df['schools'][1492])-1]
sa = json.loads(ss)
sa

{'rating': ['9/10', '2/10', '7/10'],
 'data': {'Distance': ['1.01mi', '2.35mi', '1.85mi'],
  'Grades': ['K-5', '6-8', '9-12']},
 'name': ['Coeur D`Alene Avenue Elementary School',
  'Marina Del Rey Middle School',
  'Venice Senior High School']}

In [35]:
# Для проверки правильности парсинга признака 'schools' в json формат
for i in range(0,len(df)):
    ss = df['schools'].loc[i][1:len(df['schools'][i])-1]
    try:
        sa = json.loads(ss)
    except:
        print(i)

In [36]:
def app_schools(x,k):
    js = json.loads(x[1:len(x)-1])
    if k==0:
        return js['rating']
    if k==1:
        return js['data']['Distance']
    if k==2:
        return js['data']['Grades']
    if k==3:
        return js['name']

In [37]:
cols_schools = ['schools_rating', 'schools_Distance', 'schools_Grades', 'schools_name']
df[cols_schools] = ''

In [38]:
df['schools_rating'] = df['schools'].apply(lambda x: app_schools(x,0))

In [39]:
df['schools_Distance'] = df['schools'].apply(lambda x: app_schools(x,1))

In [40]:
df['schools_Grades'] = df['schools'].apply(lambda x: app_schools(x,2))

In [41]:
df['schools_name'] = df['schools'].apply(lambda x: app_schools(x,3))

In [52]:
df['schools_rating']

0         [4, 4, 7, NR, 4, 7, NR, NR]
1                  [4/10, 0/10, 4/10]
2                  [8/10, 4/10, 8/10]
3           [9/10, 9/10, 10/10, 9/10]
4                  [4/10, 5/10, 5/10]
                     ...             
374699                  [10/10, 5/10]
374700             [1/10, 5/10, 7/10]
374701                   [5/10, 4/10]
374702                   [NA, NA, NA]
374703             [5/10, 4/10, 3/10]
Name: schools_rating, Length: 374704, dtype: object

In [45]:
df['schools_name']

0         [Southern Pines Elementary School, Southern Mi...
1         [East Valley High School&Extension, Eastvalley...
2         [Paul Revere Middle School, Brentwood Science ...
3         [Mcculloch Intermediate School, Bradfield Elem...
4         [Southwest Middle School, Bayside High School,...
                                ...                        
374699    [Air Base Elementary School, Dr Michael M. Kro...
374700    [Hope College Prep High School, Lake View High...
374701    [Ps 48 William Wordsworth, Jhs 8 Richard S Gro...
374702    [Hiliard El, Forest Brook Middle, North Forest...
374703    [Mary Lou Hartman, Woodlake Hills Middle Schoo...
Name: schools_name, Length: 374704, dtype: object

#### Признак "status"

In [61]:
df['status'].unique()

array(['Active', 'for sale', nan, 'New construction', 'New', 'For sale',
       'Pending', 'P', 'Active/Contingent', ' / auction',
       'Under Contract', 'Pre-foreclosure / auction',
       'Under Contract   Showing', 'Pre-foreclosure',
       'Under Contract Backups', 'Active Under Contract', 'foreclosure',
       'Foreclosed', 'Option Pending', 'Under Contract Show', 'for rent',
       'Auction', 'A Active', 'Contingent', 'Pending   Continue To Show',
       'Price Change', 'Back on Market', 'Active Option', 'Foreclosure',
       'Coming soon: Nov 21.', 'Contingent Finance And Inspection',
       'Coming soon: Dec 4.', 'P Pending Sale', 'Coming soon: Nov 23.',
       'Active With Contingencies', 'Pending Ab', 'Pf', 'Contingent Show',
       'Contract P', 'Contingent Take Backup', 'Apartment for rent',
       'Backup Contract', 'Option Contract', 'Pending Continue To Show',
       'pending', 'Pending Inspection', 'Active Option Contract', 'C',
       'Auction - Active', 'Contingent 

In [62]:
df['status'].nunique()

156

In [63]:
df['status'].value_counts()

for sale                 156100
Active                   105206
For sale                  43465
foreclosure                5677
New construction           5459
                          ...  
Contingent   No Show          1
Pending Take Backups          1
Conting Accpt Backups         1
Backup                        1
Coming soon: Oct 21.          1
Name: status, Length: 156, dtype: int64

In [66]:
def make_status(x):
    if x.startswith('Coming soon'): x = 'coming soon'
    x = x.lower()
    x = re.sub('[^a-z]', ' ', x)
    x = re.sub(r'\b\w{,2}\b', '', x)
    x = re.sub(r'\s+', ' ', x)
    return x.strip()

In [64]:
df['status'] = df['status'].fillna('')

In [67]:
df['status'] = df['status'].apply(make_status)

In [68]:
df['status'].value_counts()

for sale                 199567
active                   105649
                          40831
foreclosure                6020
new construction           5459
                          ...  
pending bring backup          1
conditional contract          1
pending take backups          1
temporary active              1
pending backup wanted         1
Name: status, Length: 83, dtype: int64

In [69]:
df['status'].unique()

array(['active', 'for sale', '', 'new construction', 'new', 'pending',
       'active contingent', 'auction', 'under contract',
       'pre foreclosure auction', 'under contract showing',
       'pre foreclosure', 'under contract backups',
       'active under contract', 'foreclosure', 'foreclosed',
       'option pending', 'under contract show', 'for rent', 'contingent',
       'pending continue show', 'price change', 'back market',
       'active option', 'coming soon',
       'contingent finance and inspection', 'pending sale',
       'active with contingencies', 'contingent show', 'contract',
       'contingent take backup', 'apartment for rent', 'backup contract',
       'option contract', 'pending inspection', 'active option contract',
       'auction active', 'due diligence period',
       'contract contingent buyer sale', 'active contingency',
       'pending with contingencies', 'lease purchase pending',
       'pending bring backup', 'continue show', 'pending taking backups',

In [70]:
df['status'].nunique()

83

#### Признак "state"

In [71]:
df['state'].value_counts()

FL    114667
TX     83358
NY     24339
CA     23208
NC     21786
TN     18231
WA     13739
OH     12432
IL      8852
NV      8421
GA      6641
CO      6377
PA      5510
MI      5126
DC      4585
AZ      3347
IN      3294
OR      2778
MA      1498
UT      1321
MD      1086
VT       864
MO       832
VA       800
WI       452
NJ       436
ME       258
IA       242
KY        90
OK        49
MS        40
SC        28
MT         7
DE         5
Fl         1
AL         1
OT         1
BA         1
OS         1
Name: state, dtype: int64

In [72]:
df['state'] = df['state'].apply(lambda x: x.upper())
df['state'].unique()

array(['NC', 'WA', 'CA', 'TX', 'FL', 'PA', 'TN', 'IA', 'NY', 'OR', 'DC',
       'NV', 'AZ', 'GA', 'IL', 'NJ', 'MA', 'OH', 'IN', 'UT', 'MI', 'VT',
       'MD', 'CO', 'VA', 'KY', 'MO', 'WI', 'ME', 'MS', 'OK', 'SC', 'MT',
       'DE', 'BA', 'AL', 'OT', 'OS'], dtype=object)

#### Признак "PrivatePool" & "private pool"

In [73]:
df['private pool'].value_counts()

Yes    4151
Name: private pool, dtype: int64

In [74]:
df['PrivatePool'].value_counts()

yes    28687
Yes    11434
Name: PrivatePool, dtype: int64

In [75]:
df['private pool'] = df['private pool'].apply(lambda x: 1 if x=='Yes' else 0)
df['PrivatePool'] = df['PrivatePool'].apply(lambda x: 1 if (x=='Yes' or x=='yes') else 0)

In [76]:
df['PrivatePool'].value_counts()

0    334583
1     40121
Name: PrivatePool, dtype: int64

In [77]:
df['private pool'].value_counts()

0    370553
1      4151
Name: private pool, dtype: int64

In [78]:
# Объединим два признака:
df['PrivatePool'] = df['private pool'] | df['PrivatePool']
df.drop(['private pool'], axis=1, inplace=True)

In [79]:
df['PrivatePool'].value_counts()

0    330432
1     44272
Name: PrivatePool, dtype: int64

#### Признак "propertyType"

In [80]:
df['propertyType'] = df['propertyType'].fillna(' ')

In [81]:
df['propertyType'] = df['propertyType'].apply(lambda x: x.lower())

In [82]:
df['propertyType'].value_counts()

single-family home                                            91376
single family                                                 61888
condo                                                         42372
                                                              34678
single family home                                            31725
                                                              ...  
1 story, contemporary, spanish, traditional, mediterranean        1
garden/zero lot line, single detached, craftsman                  1
farm house, traditional                                           1
dutch, contemporary                                               1
3+ stories, tudor, historic/older, traditional                    1
Name: propertyType, Length: 1271, dtype: int64

In [83]:
df['propertyType'].value_counts().index.tolist()

['single-family home',
 'single family',
 'condo',
 ' ',
 'single family home',
 'lot/land',
 'townhouse',
 'land',
 'multi-family',
 'condo/townhome/row home/co-op',
 'traditional',
 'coop',
 'multi family',
 'high rise',
 'ranch',
 'detached, one story',
 'mobile/manufactured',
 'single detached, traditional',
 'contemporary',
 'multi-family home',
 '1 story',
 'colonial',
 'mobile / manufactured',
 'contemporary/modern',
 '2 stories',
 'apartment',
 'mfd/mobile home',
 'single detached',
 'detached, two story',
 'one story',
 'transitional',
 'cooperative',
 'florida',
 'bungalow',
 'farms/ranches',
 'two story',
 'garden home',
 'cape cod',
 '2 story',
 '2 stories, traditional',
 'spanish/mediterranean',
 'other style',
 '1 story, traditional',
 'other',
 'condo/townhome, traditional',
 'craftsman',
 'contemporary/modern, traditional',
 'condo/townhome',
 'condo/townhome, contemporary/modern',
 'singlefamilyresidence',
 'single detached, contemporary/modern',
 'multiple occupancy',

In [84]:
old = ['one story', 'two story', '2-story', '2 stories', '1 1/2 story', '3+ stories', 'multi family', 'single family', 
       'singlefamilyresidence', 'townhome', 'lot/land', 'split level', 'bi level', 'bi-level', 'bilevel', 'high rise',  
       'hi-rise', 'condo/unit',  'condo', 'other', 'mobile / manufactured', 'manuf/mobile', 'co-op',
       'coop', 'cape cod', 'modernist']
new = ['1 story',  '2 story', '2 story', '2 story',  '1.5 story', '3+ story', 'multi-family', 'single-family',
       'single-family', 'townhouse', 'land', 'split-level', 'split-level', 'split-level', 'split-level', 'high-rise', 
       'high-rise',  'condominium', 'condominium','other style', 'mobile/manufactured', 'mobile/manufactured', 'cooperative', 
       'cooperative',  'cape', 'modern']
for i in range(0,len(new)):
    df['propertyType'] = df['propertyType'].apply(lambda x: x.replace(old[i], new[i]))
    df['propertyType'] = df['propertyType'].apply(lambda x: x.replace('cape', 'cape cod'))

new = ['english', 'georgian', 'florida','spanish','ranch', '1 story',  '1.5 story', '2 story','3+ story', 'townhouse', 
       'single-family','multi-family',  'land', 'single detached', 'high-rise', 'condominium', 'contemporary', 
       'other style', 'bungalow', 'craftsman',  'split-level', 'cape cod','cooperative', 'a-frame',  'custom', 'loft', 
       'multi-level', 'attached']
for i in range(0, len(new)):
    df['propertyType'] = df['propertyType'].apply(lambda x: new[i] if (-1 != x.find(new[i])) else x)

In [85]:
Type = df['propertyType'].value_counts()
for i in range(0, 100):
    print(Type.index[i])

single-family
condominium
 
land
townhouse
multi-family
traditional
1 story
cooperative
contemporary
ranch
single detached
2 story
mobile/manufactured
high-rise
colonial
apartment
mfd/mobile home
florida
other style
spanish
bungalow
transitional
cape cod
garden home
split-level
craftsman
custom
multiple occupancy
mediterranean
traditional, transitional
cluster home
federal
victorian
residential (<1 acre)
low-rise (1-3 stories)
miscellaneous
detached
attached
tri-level
english
key west
penthouse
tudor
a-frame
cottage
duplex
fourplex
georgian
straight thru
mid-century modern
modern
santa barbara/tuscan
multi-level
3+ story
farmhouse
multiplex
mediterranean, traditional
french
colonial, traditional
1.5 story
loft
unknown
french, traditional
mid-rise
european
elevated
beaux arts
coach/carriage
residential (1+ acre)
french provincial
garden apartment
farm house
historical
cluster home, garden home
commercial/industrial
art deco
mid-rise (4-7 stories)
charleston
split-entry
less than 4 floor

In [86]:
df['propertyType'].value_counts()

single-family                     185168
condominium                        42797
                                   34678
land                               31477
townhouse                          27042
                                   ...  
french, tudor                          1
side-by-side, colonial                 1
colonial, converted dwelling           1
farm house, traditional                1
lake house, mid-century modern         1
Name: propertyType, Length: 291, dtype: int64

In [88]:
Type.index[27]

'custom'

In [89]:
Type[27]

156

#### Признак "fireplace"

In [90]:
df['fireplace'] = df['fireplace'].fillna('0') # Заполнение 0 пустых признаков, отсутствие камин 
df['fireplace'] = df['fireplace'].apply(lambda x: x.lower()) #  Перевод всех символов к нижнему регистру

In [91]:
no = ['no fireplace', 'no', 'n/k'] # Признаки  отсутствия камин
for i in range(0, len(no)):
    df['fireplace'] = df['fireplace'].apply(lambda x: '0' if x==no[i] else x)

In [92]:
df['fireplace'] = df['fireplace'].apply(lambda x: '1' if x=='yes' else x) # Есть камин, считаем,  что единственный

In [93]:
# Для подсчета камин в доме используем слово "room". 
# Для этого в названиях комнат расшифруем сокращения и пропущенное слово "room". заодно расшифруем слово 'fp'
old= ['fp', 'bedroom',  'familyrm', 'family rm', 'livingrm', 'living rm', 'guest house',
      'kitchen',  'basement',  'in-law quarters'] 

new = ['fireplace',   'bed room', 'family room', 'family room', 'living room', 'living room', 'guest house room',
'kitchen room', 'basement room',  'in-law quarters room']

for i in range(0, len(old)):
    df['fireplace'] = df['fireplace'].apply(lambda x: x.replace(old[i], new[i]))

In [94]:
# Здесь слово из yes_substr заменяется на слово из in_str , если отсуствует некотроые сопутствующее слово из non_substr.
yes_substr = ['master', 'dining', 'den']
non_substr = ['bed room', 'kitchen/dining room', 'den/family']
in_str = ['master bed room', 'dining room', 'den room'] 
for i in range(0,len(yes_substr)):
    df['fireplace'] = df['fireplace'].apply(
        lambda x: x.replace(yes_substr[i], in_str[i]) if ((-1!=x.find(yes_substr[i])) and (-1==x.find(non_substr[i]))) else x)


In [95]:
#  Переводим словесное описания чисел в цифровой вид и заменим всю строку
n_old = ['one', 'two', 'three', 'four'] 
n_new = ['1','2', '3', '4']
for i in range(0, len(n_old)):
    df['fireplace'] = df['fireplace'].apply(lambda x: n_new[i] if (-1!=x.find(n_old[i])) else x)

In [96]:
sub_str = ['fireplace','gas log']
for i in range(0,len(sub_str)):
    df['fireplace'] = df['fireplace'].apply(lambda x: '1' if -1!=x.find(sub_str[i]) else x)

In [97]:
# Подсчет количества комнат, где имеется камин
df['fireplace'] = df['fireplace'].apply(lambda x: str(x.count('room'))+' '+x if 0<x.count('room') else x)

df['fireplace'] = df['fireplace'].apply(
        lambda x: '2' if (-1!=x.find('2') and  -1==x.find('2+') and -1==x.find('12')) else x)

df['fireplace'] = df['fireplace'].apply(
        lambda x: '1' if (-1!=x.find('1') and -1==x.find('10') and -1==x.find('11') and -1==x.find('12')) else x)

In [98]:
# Замена всю строку на элемент из n_new  при отсутсвие сопутствущего слово из n_not
n_new = ['3', '4', 'ceiling fan']
n_not = ['3+', '4+', 'fireplace']
for i in range(0,len(n_new)):
    df['fireplace'] = df['fireplace'].apply(
        lambda x: n_new[i] if (-1!=x.find(n_new[i]) and -1==x.find(n_not[i])) else x)

In [99]:
n = ['2+', '3+', '4+', '5', '6', '7', '8', '9' , '10', '11', '12']
for i in range(0, len(n)):
    df['fireplace'] = df['fireplace'].apply(lambda x: n[i] if (-1!=x.find(n[i])) else x)

In [100]:
df['fireplace'] = df['fireplace'].apply(lambda x: '0' if -1!=x.find('utility connection') else x)

In [101]:
df['fireplace'].value_counts()

0                                      272938
1                                       92554
2                                        2789
not applicable                           1990
ceiling fan                              1173
                                        ...  
pellet                                      1
extra closets, redecorated, storage         1
wood burning, includes accessories          1
gas, attached doors, flue                   1
gas burning, woodburning                    1
Name: fireplace, Length: 243, dtype: int64

In [103]:
Type = df['fireplace'].value_counts()
s = Type.index.tolist()
s

['0',
 '1',
 '2',
 'not applicable',
 'ceiling fan',
 '3',
 'location',
 'wood burning',
 'special features',
 'wood',
 '4',
 'gas',
 'decorative',
 'electric',
 '5',
 'gas fuel,location',
 'walk-in closets',
 '6',
 '3+',
 'firepit',
 'other (see remarks)',
 'wood burning, gas starter',
 'gas starter',
 'factory built',
 'gas, ventless',
 '7',
 'extra closets',
 'gas fuel',
 'fire pit,location',
 'decorative,location',
 'extras',
 'storage, walk-in closets',
 'insert',
 'wood fuel,location',
 'storage',
 'wood stove',
 'masonry',
 '4+',
 'woodstove',
 'see agent',
 'brick',
 'decorative, electric',
 'wood burning frplc',
 'fire pit,gas fuel,location',
 'woodburning stove',
 'free standing',
 'extra closets, storage, walk-in closets',
 'smthr',
 'natural gas',
 'elevator, storage, walk-in closets',
 'smapl',
 'gas starters',
 'gas, non wood burning',
 'gas burning',
 'gas fuel,gas starter,location',
 'wood fuel',
 'gas vented',
 '8',
 'log wood-burning',
 'number',
 'log woodburning',
 

In [105]:
# Строк датасет, где предполагается отсутствие камина ('fireplace')
fp_no =['not applicable',
 'ceiling fan',
 'smthr',
 'smapl',
 'number',
 'extra closets, walk-in closets',
 'extra closets, smthr, storage, walk-in closets',
 'redecorated',
 'other-see remarks',
 'air filter, storage, walk-in closets',
 'extra closets, smthr, walk-in closets',
 'smthr, storage, walk-in closets',
 'air filter',
 'see through',
 'elevator',
 'extra closets, smthr',
 'storage, wet bar',
 'extra closets, redecorated, storage',
 'fire sprinkler system, smcam, smlt, smthr, storage, walk-in closets',
 'fire sprinkler system, redecorated',
 'central vacuum, elevator, extra closets, walk-in closets, wet bar',
 'water filter',
 'air filter, extra closets, smthr, walk-in closets',
 'heating system: yes(s)',
 'water filter, wet bar',
 'central vacuum, elevator, walk-in closets, wet bar',
 'smcam',
 'extra closets, fire sprinkler system',
 'central vacuum, smthr',
 'extra closets, storage, walk-in closets, water filter, wet bar',
 'redecorated, storage',
 'air filter, dehumidifier, extra closets, redecorated, storage, water filter',
 'fire sprinkler system, smthr',
 'extra closets, storage, walk-in closets, wet bar',
 'extra closets, fire sprinkler system, redecorated',
 'air filter, walk-in closets, water filter, wet bar',
 'central vacuum, elevator, extra closets, hot tub, smthr, wet bar',
 'extra closets, smcam, smthr',
 'elevator, fire sprinkler system, storage, walk-in closets',
 'walk-in closets, water filter, wet bar',
 'hot tub, redecorated',
 'redecorated, walk-in closets',
 'extra closets, smapl',
 'presently inoperative, decorative only',
 'other - see remarks',
 'air filter, central vacuum, intercom, walk-in closets',
 'firepit',
 'fire pit,gas fuel,location',
 'fire pit, location',
 'firepit, wood',
 'central vacuum',
 'central vacuum, elevator, extra closets, storage, walk-in closets, wet bar',
 'storage, walk-in closets, water filter, wet bar',
 'fire pit,see through,location',
 'air filter, central vacuum, elevator, extra closets, redecorated, wet bar',
 'air filter, elevator, extra closets, walk-in closets, wet bar',
 'elevator, walk-in closets',
 'central vacuum, extra closets, smthr, storage, walk-in closets',
 'elevator, extra closets, fire sprinkler system, storage, walk-in closets',
 'air filter, extra closets, storage, walk-in closets',
 'dehumidifier, fire sprinkler system, storage, walk-in closets, water filter']

f = open("fireplace_no", "w")
for  line in fp_no:
    f.write(line)
    f.write('\n')
f.close()

In [106]:
s1 = list(set(s) - set(fp_no))
s1

['gas unvented, propane',
 'walk-in closets, wet bar',
 '8',
 'vented gas',
 'electric,location',
 'decorative',
 'extra closets, fire sprinkler system, storage, walk-in closets',
 'brick, flue, wood burning stove',
 'free standing, propane',
 'attached doors, ventless',
 'gas, attached doors, flue',
 'decorative only',
 'extra closets, storage',
 'extra closets, fire sprinkler system, redecorated, storage',
 'walk-in closets',
 'propane stove, freestanding',
 'gas fuel,gas starter',
 'wood burning, woodstove',
 'propane, wood burning',
 'gas burning, woodburning',
 'other (see remarks)',
 'coal',
 'gas unvented',
 'factory built, metal',
 'outside, wood burning',
 'insert, recirculator, wood',
 'wood burning, gas',
 'insert, wood',
 'log wood-burning',
 'extra closets, storage, walk-in closets',
 'woodburning, gas starter, living, freestanding',
 'wood stove insert',
 'wood burning, heatilator type system, includes accessories',
 'gas',
 'brick',
 'gas vented, natural gas',
 'gas, gas

In [107]:
fp_yes = ['natural gas, propane logs convey',
 'elevator, storage, walk-in closets',
 'wood burning, gas',
 'wood burning w/gas starter',
 'bath features',
 'redecorated, storage, walk-in closets',
 'decorative,fire pit,location',
 'recirculator, wood',
 'gas fuel,wood fuel,location',
 'factory built, gas starter',
 'wood, location',
 'wood burning',
 'insert unit, blower',
 'fire sprinkler system',
 'extra closets, fire sprinkler system, storage, walk-in closets',
 'gas vented',
 'wood stove',
 'attached doors, ventless',
 'factory built, metal, wood stove',
 'insert, wood',
 'extra closets, storage, walk-in closets',
 'wood burning, woodstove',
 'fire pit,free standing,electric,location',
 'outside, wood burning',
 'woodburning',
 'wood fuel,location',
 'gas, wood burning',
 'wood burning, includes accessories',
 'brick, wood burning, wood stove',
 'factory built, metal',
 'central vacuum, extra closets, redecorated, smthr, walk-in closets',
 'electric logs',
 'free standing,location',
 'ventless, propane',
 'wood',
 'gas starter',
 'prefab',
 'free-standing, insert, wood',
 'flue',
 'gas, location',
 'gas, propane logs convey',
 'electric, wood burning',
 'gas',
 'gas, ventless, propane',
 'gas, wall mounted',
 'woodburning, gas starter, living, freestanding',
 'gas starter, portable/ventless',
 'metal',
 'insert, recirculator, wood',
 'decorative, electric, wood burning',
 'decorative',
 'free standing',
 'wood burning, heatilator type system, includes accessories',
 'fire pit,location',
 'ventless',
 'free standing, wood burning',
 'factory built, wood stove',
 'free standing, propane',
 'bath',
 'wood burning, electric, gas starter',
 'in living area, wood burning',
 'living',
 'gas, attached doors, flue',
 'wood burning, gas starters, ventless',
 'wall mounted, electric',
 'decorative,gas fuel',
 'gas burning, woodburning',
 'gas unvented, propane',
 'gas starters',
 'gas, gas starters',
 'coal',
 'electric logs, outside',
 'free standing wood stove',
 'insert',
 'decorative, wood burning',
 'decorative, electric, non wood burning',
 'other (see remarks)',
 'intercom, storage',
 'propane stove, freestanding',
 'wood fuel',
 'marble, stove',
 'decorative, gas',
 'wood burning frplc',
 'gas fuel,location',
 'gas starter, masonry',
 'extra closets, redecorated, storage, walk-in closets',
 'wet bar',
 'see through,location',
 'factory built',
 'gas, gas starters, free standing',
 'propane, wood burning',
 'vented gas',
 'decorative only',
 'gas, propane',
 'gas, ventless',
 'special features',
 'extra closets, fire sprinkler system, redecorated, storage',
 'decorative,location',
 'gas burning',
 'extra closets, smcam, smlt, smthr, storage, wet bar',
 'unit features',
 'extras',
 'extra closets, storage',
 'woodstove',
 'wood stove insert',
 'smcam, smlt, smthr, storage, walk-in closets',
 'brick',
 'decorative, electric, free standing',
 'walk-in closets, wet bar',
 'decorative,gas fuel,location',
 'outdoors',
 'gas unvented',
 'brick, flue, wood burning stove',
 'brick, wood burning',
 'in living area',
 'wood burning, electric logs',
 'log wood-burning',
 'decorative, other (see remarks)',
 'free-standing, wood',
 'location',
 'stove, wood burning',
 'see agent',
 'extra closets',
 'gas vented, natural gas',
 'pellet',
 'electric',
 'masonry',
 'pre-fab',
 'decorative, electric',
 'woodburning stove',
 'smthr, walk-in closets',
 'gas fuel,wood fuel',
 'gas, gas starters, attached screens',
 'fire sprinkler system, smlt, smthr',
 'pellet, wood',
 'walk-in closets',
 'wood burning, gas, flue',
 'log woodburning',
 'gas fuel',
 'gas fuel,gas starter,location',
 'gas, attached screens, ventless',
 'wood-burning stove',
 'storage',
 'gas, wood stove, ventless',
 'heatilator type system',
 'free standing frplc',
 'wood burning, gas starter',
 'pellet stove',
 'masonry, wood stove',
 'electric, non wood burning',
 'electric, free standing',
 'fire pit',
 'gas, gas starters, ventless',
 'brick, free standing',
 'gas, gas starters, ventless, propane',
 'in living area, wood burning, free standing',
 'wood burning stove',
 'direct vent',
 'gas, attached screens',
 'storage, walk-in closets',
 'electric logs, free standing',
 'gas fuel,gas starter',
 'factory built, masonry',
 'electric,location',
 'study',
 'natural gas',
 'gas, non wood burning',
 'electric,gas fuel,location']

f = open("fireplace_yes", "w")
for  line in fp_yes:
    f.write(line)
    f.write('\n')
f.close()

In [108]:
#  Перемменное содежит строк, по смыслу указывающие наличия одного камина. 
f = open("fireplace_no", "r")
fp_0 = f.readlines()
f.close()

In [109]:
#  Перемменное содежит строк, по смыслу указывающие наличия одного камина. 
f = open("fireplace_yes", "r")
fp_1 = f.readlines()
f.close()

In [110]:
for i in range(0, len(fp_0)):
    df['fireplace'] = df['fireplace'].apply(lambda x: '0' if x== fp_0[i].strip() else x)
for i in range(0, len(fp_1)):
    df['fireplace'] = df['fireplace'].apply(lambda x: '1' if x== fp_1[i].strip() else x)

In [111]:
df['fireplace'].value_counts()

# Признак 'fireplace' - категориальный

0     276228
1      94679
2       2789
3        628
4        198
5         70
6         36
3+        34
7         18
4+        10
8          5
10         3
9          3
11         2
12         1
Name: fireplace, dtype: int64

In [ ]:
Признак оказался самым сложным. Его можно сделать категориальным, или в другом варианте модели сделаем числовым.

#### Признак "city"

In [112]:
df['city'].value_counts()

Houston               24407
San Antonio           15501
Miami                 15411
Jacksonville           9892
Dallas                 8771
                      ...  
Hyden                     1
Tekonsha                  1
Mount Gilead              1
Palm Beach Gardens        1
Blue Springs              1
Name: city, Length: 2019, dtype: int64

In [113]:
df['city'].value_counts().index[3]

'Jacksonville'

In [114]:
df['city'].value_counts()[3]

9892

In [115]:
df['city'] = df['city'].fillna('other')
top_city = df['city'].value_counts()[:200].index

In [ ]:
df['city'] = df['city'].apply(lambda r: r if r in top_city else 'other')

In [116]:
df['city'].value_counts()

other           59425
Houston         24407
San Antonio     15501
Miami           15411
Jacksonville     9892
                ...  
Greenville        277
Palmetto Bay      276
Surfside          273
Dunnellon         273
Huntley           273
Name: city, Length: 201, dtype: int64

#### Признак "sqft"

In [ ]:
# df['sqft'] = data['sqft']

In [117]:
df['sqft'].value_counts()

0                                          11853
1,200 sqft                                   829
1,000 sqft                                   648
1,100 sqft                                   570
1,800 sqft                                   562
                                           ...  
4689                                           1
Total interior livable area: 5,400 sqft        1
15,842 sqft                                    1
Total interior livable area: 4,826 sqft        1
4,479 sqft                                     1
Name: sqft, Length: 25373, dtype: int64

In [118]:
def make_features_sqft(x):
    if x == -1.0: return x
    x = re.sub('[^0-9]', '', x)
    return x

In [119]:
df.sqft = df.sqft.fillna(-1.0)
df.sqft = df.sqft.apply(make_features_sqft)

In [120]:
df['sqft'] = df['sqft'].apply(lambda x: -1.0 if x=='' or x=='0' else x)

In [121]:
df['sqft'].astype(np.float)

0         2900.0
1         1947.0
2         3000.0
3         6457.0
4           -1.0
           ...  
374699    4017.0
374700    2000.0
374701    1152.0
374702      -1.0
374703    1462.0
Name: sqft, Length: 374704, dtype: float64

#### Признак "baths"

In [122]:
def make_baths(x):
    x = re.sub('ba','Baths', x)
    x = re.sub('1 1/2', '1.5', x)
    x = x.replace('.0', '')
    x = x.replace('Bathsths','Baths')
    if x == 'Bathrooms: 241': 
        x='Bathrooms: 4'
    if x == 'Bathrooms: SemiMod': 
        x='Bathrooms: 6'
    if x=='1 / 1-0 / 1-0 / 1-0' or x=='2-1 / 2-1 / 1-1 / 1-1':  
        x = '2'
    if x=='1 / 1 / 1 / 1' or x=='1-0 / 1-0 / 1' or x=='3-1 / 2-2':  
        x = '3'
    if x=='116 / 116 / 116':
        x='116'
    if x=='0 / 0' or x== '0.00' or x=='0' or x=='0.0' or x=='00' or x=='0 /':
        x=' '
    if (-1.0!=x.find('--') or -1.0!=x.find('~') or -1.0!=x.find('—')):
        x = ' '
    if len(x)<5 and -1==x.find('+'):
        x = x + ' Baths'
    return x

In [123]:
df.baths = df.baths.fillna(' ')
df.baths = df.baths.apply(make_baths)

In [124]:
df.baths.value_counts()

  Baths          110572
2 Baths           91550
3 Baths           58175
4 Baths           23579
1 Baths           13115
                  ...  
41 Baths              1
43 Baths              1
11.5 Baths            1
Bathrooms: 21         1
39 Baths              1
Name: baths, Length: 128, dtype: int64

#### Признак "beds"

In [125]:
df['beds'].value_counts()

3 Beds        53077
4 Beds        35225
3             31191
2 Beds        26196
4             19915
              ...  
5.92 acres        1
3,968 sqft        1
3.58 acres        1
9,461 sqft        1
8.98 acres        1
Name: beds, Length: 1147, dtype: int64

In [126]:
df.beds = df.beds.fillna(' ')

In [127]:
df['beds'].value_counts().index.tolist()

[' ',
 '3 Beds',
 '4 Beds',
 '3',
 '2 Beds',
 '4',
 '2',
 'Baths',
 '3 bd',
 '5 Beds',
 '4 bd',
 '3.0',
 '5',
 '4.0',
 '2 bd',
 '1',
 '6 Beds',
 '5 bd',
 '2.0',
 '6',
 '5.0',
 '0',
 '7 Beds',
 '1 bd',
 '-- bd',
 '8 Beds',
 'Bath',
 '6 bd',
 '7',
 '1.0',
 '9 Beds',
 '8',
 '6.0',
 '0.0',
 '10 Beds',
 '7 bd',
 '12 Beds',
 '9',
 '8 bd',
 '-- sqft',
 '5,000 sqft',
 '11 Beds',
 '7.0',
 '10',
 '1 acre',
 '64 Beds',
 '8.0',
 '16 Beds',
 '0.34 acres',
 '0.28 acres',
 '11',
 '9 bd',
 '12',
 '0.26 acres',
 '0.29 acres',
 '13 Beds',
 '1.03 acres',
 '10,001 sqft',
 '0.3 acres',
 '0.27 acres',
 '14 Beds',
 '2,874 sqft',
 '0.46 acres',
 '0.5 acres',
 '0.43 acres',
 '18 Beds',
 '15 Beds',
 '10 bd',
 '0.37 acres',
 '12 bd',
 '6,098 sqft',
 '8,712 sqft',
 '16',
 '0.39 acres',
 '0.25 acres',
 '0.33 acres',
 '0.35 acres',
 '7,405 sqft',
 '0.41 acres',
 '10,018 sqft',
 '0.32 acres',
 '0.36 acres',
 '24 Beds',
 '0.4 acres',
 '0.47 acres',
 '0.53 acres',
 '5,009 sqft',
 '0.55 acres',
 '0.52 acres',
 '0.31 ac

In [128]:
def make_beds(x):
    x = x.replace('bd','Beds')
    x = x.replace('.0', '')
    x = x.replace('--', ' ')
    if len(x)<=3:
        x = x + ' Beds'
    return x

In [129]:
df.beds = df.beds.apply(make_beds)

In [130]:
df['beds'].value_counts()

3 Beds                  105031
  Beds                   92151
4 Beds                   68564
2 Beds                   49789
5 Beds                   21610
                         ...  
5.92 acres                   1
# Bedrooms 1st Floor         1
3,968 sqft                   1
3.58 acres                   1
8.98 acres                   1
Name: beds, Length: 1061, dtype: int64

In [131]:
df['acres'] = +df['beds'].apply(lambda x: x if -1!=x.find('acre') else ' ')

In [132]:
ind_1 =  df[df['acres']!=' '].index

In [133]:
df['beds_sqft'] = df['beds'].apply(lambda x: x if -1!=x.find('sqft') else ' ')

In [134]:
ind_2 =  df[df['beds_sqft']!=' '].index

In [135]:
df.drop(ind_1, inplace=True)

df.drop(ind_2, inplace=True)

df = df.reset_index(drop=True)

In [136]:
cols_drop = ['acres','beds_sqft']
df.drop(cols_drop, axis=1, inplace=True)

In [137]:
df['beds'].value_counts()

3 Beds                                                                       105031
  Beds                                                                        92151
4 Beds                                                                        68564
2 Beds                                                                        49789
5 Beds                                                                        21610
                                                                              ...  
1 Bath, 2 Bedrooms, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator         1
# Bedrooms 1st Floor                                                              1
3 or More Bedrooms                                                                1
31 Beds                                                                           1
88 Beds                                                                           1
Name: beds, Length: 69, dtype: int64

In [138]:
df['beds'].value_counts().index.tolist()

['3 Beds',
 '  Beds',
 '4 Beds',
 '2 Beds',
 '5 Beds',
 'Baths',
 '6 Beds',
 '1 Beds',
 '7 Beds',
 '0 Beds',
 '8 Beds',
 'Bath',
 '9 Beds',
 '10 Beds',
 '12 Beds',
 '11 Beds',
 '16 Beds',
 '13 Beds',
 '64 Beds',
 '14 Beds',
 '15 Beds',
 '18 Beds',
 '24 Beds',
 '20 Beds',
 '17 Beds',
 '28 Beds',
 '32 Beds',
 '22 Beds',
 '26 Beds',
 '19 Beds',
 '34 Beds',
 '21 Beds',
 '23 Beds',
 '27 Beds',
 '40 Beds',
 '30 Beds',
 '44 Beds',
 '25 Beds',
 '36 Beds',
 '33 Beds',
 '42 Beds',
 '47 Beds',
 '29 Beds',
 '48 Beds',
 '78 Beds',
 '75 Beds',
 '1-2 Beds',
 '144 Beds',
 '3 or More Bedrooms, Dining Room, Living Room, Range/Oven, Refrigerator',
 '2 Baths, 3 or More Bedrooms',
 '52 Beds',
 '53 Beds',
 '39 Beds',
 '35 Beds',
 '76 Beds',
 '51 Beds',
 '99 Beds',
 "Based on Redfin's Raleigh data, we estimate the home's value is $708,248, which is 1.2% more than its current list price.",
 '1 Bath, 3 or More Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator',
 '

In [ ]:
'''
'1 Bath, 2 Bedrooms, Living Room, Range/Oven, Refrigerator',
'1 Bath, 2 Bedrooms',
'1 Bath, 2 Bedrooms, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator',
'2 Baths, 3 or More Bedrooms',
'3 or More Bedrooms',
'1 Bath, 2 Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room',
'# Bedrooms 1st Floor',
"Based on Redfin's St Johns data, we estimate the home's value is $360,731, which is 2.2% less than its current list price.",
'1 Bath, 3 or More Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator',
"Based on Redfin's Raleigh data, we estimate the home's value is $708,248, which is 1.2% more than its current list price.",
'3 or More Bedrooms, Dining Room, Living Room, Range/Oven, Refrigerator',
'''

In [140]:
df[df['beds']=='3 or More Bedrooms']

,status,propertyType,baths,homeFacts,fireplace,city,schools,sqft,beds,state,...,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft,schools_rating,schools_Distance,schools_Grades,schools_name
292363,active,multi-family,Baths,"{""atAGlanceFacts"": [{""factValue"": ""1894"", ""fac...",0,other,"[{""rating"": [""1"", ""3"", ""1"", ""3"", ""2"", ""7"", ""NR...",-1.0,3 or More Bedrooms,WI,...,,,,Outdoor Space,10890,No Info,"[1, 3, 1, 3, 2, 7, NR, NR]","[0.7 mi, 6.2 mi, 0.9 mi, 0.5 mi, 0.6 mi, 1.0 m...","[PK–8, 6–12, 9–12, PK–8, PK–8, 3–12, 9–12, PK–10]",[Carver Academy of Mathematics and Science Sch...


In [ ]:
# [18515, 122996, 170079, 320539]: 1 baths, 2 beds

In [ ]:
# 234726, 275097: 1 baths, 3+ beds

In [ ]:
# 292363,  197407: 3+ beds 

In [ ]:
# 359070: 1 beds

In [ ]:
# 124539, 263153: 

In [141]:
ind = [18515, 122996, 170079, 320539]
for i in range(0, len(ind)):
    df.loc[[ind[i]], 'baths']='1 Baths'
    df.loc[[ind[i]], 'beds']='2 beds'

In [142]:
for i in [234726,275097]:
    df.loc[[i], 'baths']='1 Baths'
    df.loc[[i], 'beds']='3+ beds'

In [143]:
df.loc[[197407], 'beds']='3+ beds'
df.loc[[292363], 'beds']='3+ beds'
df.loc[[359070], 'beds']='1 beds'

In [ ]:
# Возможно при раскрыйтия признака "homeFacts" выясняется что за жил площадь
# df.drop([124539, 263153],  inplace=True)
# df = df.reset_index(drop=True)

### Результаты EDA